In [5]:
import os
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

In [6]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:/Working/C_work/Coding/LangChain/.env/service_account.json"

In [7]:
loader = TextLoader('160-KB.txt',encoding='utf-8')
documents = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=0
)
chunks = text_splitter.split_documents(documents)

In [9]:
embedding_model = VertexAIEmbeddings(
    model_name="gemini-embedding-001",
    location="us-central1"
)

d:\Programs\anaconda\envs\LangChain\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [10]:
vector_store = InMemoryVectorStore(embedding=embedding_model)

In [11]:
print("Adding documents one by one...")
for i, doc_item in enumerate(chunks):
    try:
        if isinstance(doc_item, Document):
            doc = doc_item
            print(f"Document {i+1} is already a Document object")
        elif isinstance(doc_item, str):
            doc = Document(page_content=doc_item)
            print(f"Created Document object from string: {doc_item}")
        else:
            doc = Document(page_content=str(doc_item))
            print(f"Converted to string and created Document: {str(doc_item)}")

        vector_store.add_documents([doc])
        print(f"✓ Successfully added document {i+1}")

    except Exception as e:
        print(f"✗ Error adding document {i+1}: {str(e)}")

print(f"\nCompleted adding documents to vector store!")

Adding documents one by one...
Document 1 is already a Document object
✓ Successfully added document 1
Document 2 is already a Document object
✓ Successfully added document 2
Document 3 is already a Document object
✓ Successfully added document 3
Document 4 is already a Document object
✓ Successfully added document 4
Document 5 is already a Document object
✓ Successfully added document 5
Document 6 is already a Document object
✓ Successfully added document 6
Document 7 is already a Document object
✓ Successfully added document 7
Document 8 is already a Document object
✓ Successfully added document 8
Document 9 is already a Document object
✓ Successfully added document 9
Document 10 is already a Document object
✓ Successfully added document 10
Document 11 is already a Document object
✓ Successfully added document 11
Document 12 is already a Document object
✓ Successfully added document 12
Document 13 is already a Document object
✓ Successfully added document 13
Document 14 is already a 

In [12]:
retrievers = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Retrieve top 3 most relevant chunks
)

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system","ใช้ข้อมูลจากเอกสารในการตอบคำถามให้สั้นกระชับด้วยความสุภาพเป็นกันเอง"),
    ("human","คำถาม: {question} , ข้อมูลที่เกี่ยวข้อง: {context}")
])

In [14]:
llm = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai",
    temperature=0.8
)

In [15]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# ใช้วิธีอื่นแก้เพื่อป้องกันปัญหา RunnablePassthrough
def get_context_and_question(query):
    # Get relevant documents
    docs = retrievers.invoke(query)
    context = format_docs(docs)
    return {"context": context, "question": query}

In [16]:
rag_chain = (
    get_context_and_question
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
def show_menu():
    print("\n--- RAG Chat System ---")
    print("1. Ask a question")
    print("2. Show help")
    print("3. Exit")
    return input("Choose an option (1-3): ")

while True:
    choice = show_menu()
    
    if choice == "1":
        user_input = input("Enter your question: ")
        result = rag_chain.invoke(user_input)
        print(f"Answer: {result}")
    
    elif choice == "2":
        print("This system answers questions based on your documents.")
    
    elif choice == "3":
        print("Goodbye!")
        break
    
    else:
        print("Invalid choice. Please try again.")


--- RAG Chat System ---
1. Ask a question
2. Show help
3. Exit
Answer: ไฟล์นี้เป็นข้อมูล XML เกี่ยวกับร้านหนังสือ ซึ่งมีข้อมูลหนังสือแต่ละเล่ม เช่น ชื่อหนังสือ ผู้แต่ง ราคา และหมวดหมู่ค่ะ 😊

--- RAG Chat System ---
1. Ask a question
2. Show help
3. Exit


KeyboardInterrupt: Interrupted by user